# High-level RNN TF Example

*Modified by Jordan A Caraballo Vega (jordancaraballo)*

In [1]:
import numpy as np
import os
import sys

import warnings # make notebook more readable and nice
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

import tensorflow as tf
from common.params_lstm import *
from common.utils import *

In [2]:
# Force one-gpu
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
print("OS: ", sys.platform)
print("Python: ", sys.version)
print("Numpy: ", np.__version__)
print("Tensorflow: ", tf.__version__)
print("GPU: ", get_gpu_name())
print(get_cuda_version())
print("CuDNN Version ", get_cudnn_version())

OS:  linux
Python:  3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
Numpy:  1.17.0
Tensorflow:  1.13.1
GPU:  ['Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB', 'Tesla V100-SXM2-16GB']
CUDA Version 10.1.243
CuDNN Version  No CUDNN in this machine


In [4]:
def create_symbol(CUDNN=True, 
                  maxf=MAXFEATURES, edim=EMBEDSIZE, nhid=NUMHIDDEN, batchs=BATCHSIZE):
    word_vectors = tf.contrib.layers.embed_sequence(X, vocab_size=maxf, embed_dim=edim)
    word_list = tf.unstack(word_vectors, axis=1)
    
    if not CUDNN:
        cell = tf.contrib.rnn.GRUCell(nhid)
        outputs, states = tf.contrib.rnn.static_rnn(cell, word_list, dtype=tf.float32)
    else:
        # Using cuDNN since vanilla RNN
        from tensorflow.contrib.cudnn_rnn.python.ops import cudnn_rnn_ops
        cudnn_cell = cudnn_rnn_ops.CudnnGRU(num_layers=1, 
                                            num_units=nhid, 
                                            input_size=edim, 
                                            input_mode='linear_input')
        params_size_t = cudnn_cell.params_size()
        params = tf.Variable(tf.random_uniform([params_size_t], -0.1, 0.1), validate_shape=False)   
        input_h = tf.Variable(tf.zeros([1, batchs, nhid]))
        outputs, states = cudnn_cell(input_data=word_list,
                                     input_h=input_h,
                                     params=params)
        logits = tf.layers.dense(outputs[-1], 2, activation=None, name='output')
    return logits

In [5]:
def init_model(m, y, lr=LR, b1=BETA_1, b2=BETA_2, eps=EPS):
    # Single-class labels, don't need dense one-hot
    # Expects unscaled logits, not output of tf.nn.softmax
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=m, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer(lr, b1, b2, eps)
    training_op = optimizer.minimize(loss)
    return training_op

In [6]:
%%time
# Data into format for library
x_train, x_test, y_train, y_test = imdb_for_library(seq_len=MAXLEN, max_features=MAXFEATURES)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)
print(x_train.dtype, x_test.dtype, y_train.dtype, y_test.dtype)

Preparing train set...
Preparing test set...
Trimming to 30000 max-features
Padding to length 150
(25000, 150) (25000, 150) (25000,) (25000,)
int32 int32 int32 int32
CPU times: user 4.88 s, sys: 437 ms, total: 5.31 s
Wall time: 5.34 s


In [7]:
%%time
# Place-holders
X = tf.placeholder(tf.int32, shape=[None, MAXLEN])
y = tf.placeholder(tf.int32, shape=[None])
sym = create_symbol()


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

CPU times: user 1.43 s, sys: 832 ms, total: 2.26 s
Wall time: 15.1 s


In [8]:
%%time
model = init_model(sym, y)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

CPU times: user 2.95 s, sys: 2.36 s, total: 5.32 s
Wall time: 40 s


In [9]:
%%time
# Main training loop: 22s
correct = tf.nn.in_top_k(sym, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
EPOCHS=40
for j in range(EPOCHS):
    for data, label in yield_mb(x_train, y_train, BATCHSIZE, shuffle=True):
        sess.run(model, feed_dict={X: data, y: label})
    # Log
    acc_train = sess.run(accuracy, feed_dict={X: data, y: label})
    print(j, "Train accuracy:", acc_train)

0 Train accuracy: 0.84375
1 Train accuracy: 0.921875
2 Train accuracy: 1.0
3 Train accuracy: 1.0
4 Train accuracy: 0.984375
5 Train accuracy: 1.0
6 Train accuracy: 1.0
7 Train accuracy: 1.0
8 Train accuracy: 1.0
9 Train accuracy: 1.0
10 Train accuracy: 1.0
11 Train accuracy: 1.0
12 Train accuracy: 1.0
13 Train accuracy: 1.0
14 Train accuracy: 1.0
15 Train accuracy: 1.0
16 Train accuracy: 1.0
17 Train accuracy: 1.0
18 Train accuracy: 1.0
19 Train accuracy: 1.0
20 Train accuracy: 1.0
21 Train accuracy: 1.0
22 Train accuracy: 1.0
23 Train accuracy: 1.0
24 Train accuracy: 1.0
25 Train accuracy: 1.0
26 Train accuracy: 1.0
27 Train accuracy: 1.0
28 Train accuracy: 1.0
29 Train accuracy: 1.0
30 Train accuracy: 1.0
31 Train accuracy: 1.0
32 Train accuracy: 1.0
33 Train accuracy: 1.0
34 Train accuracy: 1.0
35 Train accuracy: 1.0
36 Train accuracy: 1.0
37 Train accuracy: 1.0
38 Train accuracy: 1.0
39 Train accuracy: 1.0
CPU times: user 3min 59s, sys: 19.6 s, total: 4min 19s
Wall time: 3min 8s


In [10]:
%%time
# Main evaluation loop: 9.19s
n_samples = (y_test.shape[0]//BATCHSIZE)*BATCHSIZE
y_guess = np.zeros(n_samples, dtype=np.int)
y_truth = y_test[:n_samples]
c = 0
for data, label in yield_mb(x_test, y_test, BATCHSIZE):
    pred = tf.argmax(sym, 1)
    output = sess.run(pred, feed_dict={X: data})
    y_guess[c*BATCHSIZE:(c+1)*BATCHSIZE] = output
    c += 1

CPU times: user 22.3 s, sys: 1.05 s, total: 23.3 s
Wall time: 22.5 s


In [11]:
print("Accuracy: ", 1.*sum(y_guess == y_truth)/len(y_guess))

Accuracy:  0.8433894230769231
